<a href="https://colab.research.google.com/github/ymoslem/datasets/blob/main/StackExchange/StackExchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Register at: https://stackapps.com/apps/oauth/register
# !curl ipecho.net/plain

secret = "your_secret_key"
key = "your_api_key"

In [ ]:
# Which site to extract questions from

site = "medicalsciences"  # "law" or "medicalsciences" etc.

In [ ]:
import requests
from requests.adapters import HTTPAdapter, Retry
from requests.exceptions import ConnectionError
from tqdm.notebook import tqdm
import json
from time import sleep

# Get questions

In [ ]:
url = "https://api.stackexchange.com/2.3/questions/"
params = {"client_secret": secret,
          "key": key,
          "site": site,
          "filter": "withbody",
          "order":"desc",
          "sort":"votes",
          "pagesize":100
          }

max_pages = 80 # 300 for law or 80 for medicalsciences

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[500, 502, 503, 504, 429])

adapter = HTTPAdapter(max_retries=retries)
session = requests.Session()

# Use the adapter for all requests to endpoints that start with this URL
session.mount('https://api.stackexchange.com/', adapter)

retrieved_data = []

for page in tqdm(range(max_pages)):
  params["page"] = page+1
  response = session.get(url, params=params)

  if response.status_code != 200:
    print("Error:", response.status_code)
    break
  else:
    current_page = response.json()
    for item in current_page['items']:
      retrieved_data.append(item)
    if current_page["has_more"] == False:
      print("No more pages")
      break

In [ ]:
len(retrieved_data)

7829

In [ ]:
import json

print(json.dumps(retrieved_data[-1], indent=4))

{
    "tags": [
        "covid-19",
        "vaccination",
        "side-effects"
    ],
    "owner": {
        "account_id": 27722897,
        "reputation": 3,
        "user_id": 26386,
        "user_type": "unregistered",
        "profile_image": "https://www.gravatar.com/avatar/511c7e442e5fbe49cdcb16d96aa8cd19?s=256&d=identicon&r=PG",
        "display_name": "Unknown user",
        "link": "https://medicalsciences.stackexchange.com/users/26386/unknown-user"
    },
    "is_answered": true,
    "view_count": 67,
    "closed_date": 1675778668,
    "accepted_answer_id": 31982,
    "answer_count": 1,
    "score": -5,
    "last_activity_date": 1685671894,
    "creation_date": 1675753683,
    "question_id": 31981,
    "link": "https://medicalsciences.stackexchange.com/questions/31981/how-bad-are-covid-vaccines",
    "closed_reason": "Not suitable for this site",
    "title": "How bad are covid vaccines?",
    "body": "<p>For the past year I've been hearing about people claiming that those 

In [ ]:
# Save file - test
with open("example.json", "w") as output:
  json.dump(retrieved_data[:10], output, indent=4)

In [ ]:
# Save the whole data
import os

# output_questions_file_name = "law.stackexchange.json"
output_questions_file_name = "medical.stackexchange.json"
directory = "/content/drive/MyDrive/data/StackExchange/MedicalSciences"
full_output_path = os.path.join(directory, output_questions_file_name)

with open(full_output_path, "w+") as output:
  json.dump(retrieved_data, output, indent=4)

# Get answers

In [ ]:
import os

# file_path = "/content/drive/MyDrive/data/StackExchange/Law/"
# file_name = "law.stackexchange.json"

file_path = "/content/drive/MyDrive/data/StackExchange/MedicalSciences/"
file_name = "medical.stackexchange.json"

data = json.load(open(os.path.join(file_path, file_name)))
questions = [question for question in data if question["is_answered"]]

print("Questions total:", len(data))
print("Questions with answers:", len(questions))

Questions total: 7829
Questions with answers: 4969


In [ ]:
url = {"url":"https://api.stackexchange.com/2.3/questions/{idx}/answers"}
params = {"client_secret": secret,
          "key": key,
          "site": site,
          "filter": "withbody",
          "order":"desc",
          "sort":"votes",
          "pagesize":100}

retries = Retry(total=5,
                backoff_factor=0.99,
                status_forcelist=[400, 429, 500, 502, 503, 504])

adapter = HTTPAdapter(max_retries=retries)
session = requests.Session()

# Use the adapter for all requests to endpoints that start with this URL
session.mount('https://api.stackexchange.com/', adapter)

questions_data = []

for question_item in tqdm(questions):
  question_id = question_item["question_id"]

  question = {}
  question["question_id"] = question_item["question_id"]
  question["tags"] = question_item["tags"]
  question["score"] = question_item["score"]
  question["license"] = question_item.get("content_license", "")
  question["title"] = question_item["title"]
  question["body"] = question_item["body"]
  question["link"] = question_item["link"]
  question["answers"] = []

  response = session.get(url["url"].format(idx=question_id), params=params)

  if response.status_code != 200:
    print("Error:", response.status_code)
    break
  else:
    current_page = response.json()
    for answer_item in current_page['items']:
      if answer_item["score"] >= 0:
        answer = {}
        answer["answer_id"] = answer_item["answer_id"]
        answer["score"] = answer_item["score"]
        answer["body"] = answer_item["body"]

        question["answers"].append(answer)

  questions_data.append(question)
  sleep(0.5)


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
print(len(questions_data))
print(json.dumps(questions_data[-1], indent=2))

In [ ]:
# Save the whole data

# output_file_name = "law.stackexchange-questions-answers.json"
output_file_name = "medical.stackexchange-questions-answers.json"

full_output_path = os.path.join(file_path, output_file_name)

with open(full_output_path, "w+") as output:
  json.dump(questions_data, output, indent=4)

In [ ]:
!ls $file_path

medical.stackexchange.json  medical.stackexchange-questions-answers.json


In [ ]:
import os
import json

# output_file_name = "law.stackexchange-questions-answers.json"
output_file_name = "medical.stackexchange-questions-answers.json"

full_output_path = os.path.join(file_path, output_file_name)

with open(full_output_path, "r") as output:
  questions_answers = json.load(output)

In [ ]:
len(questions_answers)

4969

In [ ]:
print(json.dumps(questions_answers[0], indent=4))

{
    "question_id": 52,
    "tags": [
        "eye",
        "computers",
        "lifestyle"
    ],
    "score": 112,
    "license": "CC BY-SA 3.0",
    "title": "How can I protect my eyesight when using computers?",
    "body": "<p>My job requires long periods of computer screen use. I have good eyesight (20/15) and don't wear glasses, but I definitely notice my eyes feeling fatigued at the end of the day. Sometimes when I'm short on sleep, I have trouble focusing or see slightly blurred vision.</p>\n\n<p>I want to protect my eyesight as much as possible. What are the best practices for doing so if you spend a lot of time staring at screens?</p>\n",
    "link": "https://medicalsciences.stackexchange.com/questions/52/how-can-i-protect-my-eyesight-when-using-computers",
    "answers": [
        {
            "answer_id": 85,
            "score": 79,
            "body": "<h3>20-20-20 Rule</h3>\n<p>Every 20 minutes of looking at the screen, look at something 20 feet away for 20 seconds.